In [2]:
import pandas as pd

dataframe = pd.read_csv(
    "https://docs.dagster.io/assets/iris.csv",
    names=[
        "sepal_length_cm",
        "sepal_width_cm",
        "petal_length_cm",
        "petal_width_cm",
        "species",
    ],
)

dataframe.values[0:10]

array([[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.0, 1.4, 0.2, 'Iris-setosa'],
       [4.7, 3.2, 1.3, 0.2, 'Iris-setosa'],
       [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
       [5.0, 3.6, 1.4, 0.2, 'Iris-setosa'],
       [5.4, 3.9, 1.7, 0.4, 'Iris-setosa'],
       [4.6, 3.4, 1.4, 0.3, 'Iris-setosa'],
       [5.0, 3.4, 1.5, 0.2, 'Iris-setosa'],
       [4.4, 2.9, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.1, 1.5, 0.1, 'Iris-setosa']], dtype=object)

In [30]:
from sqlalchemy import create_engine
from sqlalchemy import text

import fsspec
import os
from deltalake import write_deltalake

engine = create_engine(
    "trino://trino@localhost:8080/delta", connect_args={"user": "trino"}
)

conn = engine.connect()
cur = conn.execute(text("CREATE SCHEMA IF NOT EXISTS notebook WITH (LOCATION='s3a://lakehouse/delta/')"))

# https://s3fs.readthedocs.io/en/latest/api.html#s3fs.core.S3FileSystem
# fs = fsspec.filesystem(protocol="s3a", key="minio", secret="minio123", endpoint_url="http://localhost:9000")
# file_path = os.path.join("s3a://lakehouse/delta", "iris")
# fs.makedirs(os.path.dirname(file_path), exist_ok=True)

storage_opts = {
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true", 
    "AWS_ALLOW_HTTP": "true",
    "protocol": "s3", 
    "AWS_ACCESS_KEY_ID": "minio", 
    "AWS_SECRET_ACCESS_KEY": "minio123", 
    "endpoint_url": "http://localhost:9000"
}


write_deltalake("s3://lakehouse/delta", dataframe, mode='overwrite', storage_options=storage_opts)
# fs.put(file_path, "s3a://warehouse/delta-notebook/iris", recursive=True)

In [31]:
from sqlalchemy import text


# SHOW TABLES FROM delta.notebook;
with engine.connect() as conn:
    conn.execute(text("CALL example.system.unregister_table(schema_name => 'notebook', table_name => 'iris')"))
    conn.execute(text(f"""
        CALL delta.system.register_table(schema_name => 'notebook', table_name => 'iris', table_location => 's3a://lakehouse/delta')
    """))
    conn.commit()


DBAPIError: (trino.exceptions.TrinoQueryError) TrinoQueryError(type=INTERNAL_ERROR, name=PROCEDURE_CALL_FAILED, message="Unable to execute HTTP request: Connect to localstack:9000 [localstack/172.18.0.8] failed: Connection refused", query_id=20240221_082352_00021_gk4wx)
[SQL: 
        CALL delta.system.register_table(schema_name => 'notebook', table_name => 'iris', table_location => 's3a://lakehouse/delta')
    ]
(Background on this error at: https://sqlalche.me/e/20/dbapi)